In [ ]:
# This has our asr preprocessing, but sid's psd feature transform
basedir = '/net2/derData/affective_eeg/eeg/asr_pwr_feats_sid'
import scipy
import os
mat_feats = {}
for f in os.listdir(basedir):
    mat = scipy.io.loadmat(os.path.join(basedir, f))
    
    mat_feats[f.split("_")[0]] = {
        "features": mat['data'][0],
        "ch_names": [i[0].strip() for i in mat['ch_names'][0]]
    }

In [ ]:
import numpy as np
import os
import xml.etree.ElementTree as ET
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
import scipy.interpolate
import scipy.io
import numpy as np
import scipy.misc

import matplotlib 
import matplotlib.pyplot as plt
import math

from PIL import Image

def topo(data):

    def fig2data ( fig ):
        # draw the renderer
        fig.canvas.draw()

        # Get the RGBA buffer from the figure
        w,h = fig.canvas.get_width_height()
        buf = np.fromstring(fig.canvas.tostring_argb(), dtype=np.uint8)
        buf.shape = ( w, h, 4)

        # canvas.tostring_argb give pixmap in ARGB mode. Roll the ALPHA channel to have it in RGBA mode
        buf = np.roll ( buf, 3, axis = 2 )
        return buf

    def fig2img ( fig ):
        # put the figure pixmap into a numpy array
        buf = fig2data(fig)
        w, h, d = buf.shape
        return Image.frombytes( "RGBA", ( w ,h ), buf.tostring( ) )

    EEG_PSD = np.array(data['features'])
    # print EEG_PSD.shape

    # sensor_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2','P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
    # koord = [[1,4],[0,3],[1,3],[0.5,2.5],[0,2],[0,1],[1,0],[3,0],[4,1],[4,2],[3.5,2.5],[3,3],[4,3],[3,4]] 
    sensor_names = ['Fp1','AF3','F3','F7','FC5','FC1','C3','T7','CP5','CP1','P3','P7','PO3','O1','Oz','Pz','Fp2','AF4','Fz','F4','F8','FC6','FC2','Cz','C4','T8','CP6','CP2','P4','P8','PO4','O2']
    koord = [[1,4],[1,3.5],[1,3],[0.25,3],[0.5,2.5],[1.5,2.5],[1,2],[0,2],[0.5,1.5],[1.5,1.5],[1,1],[0.25,1],[1,0.5],[1,0],[2,0],[2,1],[3,4],[3,3.5],[2,3],[3,3],[3.75,3],[3.5,2.5],[2.5,2.5],[2,2],[3,2],[4,2],[3.5,1.5],[2.5,1.5],[3,1],[3.75,1],[3,0.5],[3,0]] 
    N_points = 200
    xy_center = (2,2)
    radius = 2

    x, y = [], []

    for i in koord:
        x.append(i[0])
        y.append(i[1])

    xi = np.linspace(-2,6,N_points)
    yi = np.linspace(-2,6,N_points)

    # For theta, alpha and beta bands
    fig_t = plt.figure()
    plt.close()
    fig_a = plt.figure()
    plt.close()
    fig_b = plt.figure()
    plt.close()

    ax_t = fig_t.add_subplot(111, aspect = 1)
    ax_a = fig_a.add_subplot(111, aspect = 1)
    ax_b = fig_b.add_subplot(111, aspect = 1)
    # ax_t.scatter(x, y, marker = 'o', c = 'b', s = 15, zorder = 3)

    # remove the ticks
    ax_t.set_xticks([])
    ax_t.set_yticks([])
    ax_a.set_xticks([])
    ax_a.set_yticks([])
    ax_b.set_xticks([])
    ax_b.set_yticks([])

    ax_t.set_xlim(0, 4)
    ax_t.set_ylim(0, 4)
    ax_a.set_xlim(0, 4)
    ax_a.set_ylim(0, 4)
    ax_b.set_xlim(0, 4)
    ax_b.set_ylim(0, 4)

    max_val = np.array([0.0,0.0,0.0])


    # For theta, alpha and beta bands
    zi_t = scipy.interpolate.griddata((x, y), EEG_PSD[0:32], (xi[None,:], yi[:,None]), method = 'cubic')
    zi_a = scipy.interpolate.griddata((x, y), EEG_PSD[32:64], (xi[None,:], yi[:,None]), method = 'cubic')
    zi_b = scipy.interpolate.griddata((x, y), EEG_PSD[64:96], (xi[None,:], yi[:,None]), method = 'cubic')

    max_val[0] = np.nanmax(zi_t)
    max_val[1] = np.nanmax(zi_a)
    max_val[2] = np.nanmax(zi_b)

    max_ind = np.argmax(max_val)
    max_val_scaled = max_val / max_val[max_ind]
    # print max_val_scaled

    CS_t = ax_t.contourf(xi, yi, zi_t, 60, cmap = plt.cm.Reds, zorder = 1)
    CS_a = ax_a.contourf(xi, yi, zi_a, 60, cmap = plt.cm.Greens, zorder = 1)
    CS_b = ax_b.contourf(xi, yi, zi_b, 60, cmap = plt.cm.Blues, zorder = 1)

    im_t = fig2img(fig_t)
    im_a = fig2img(fig_a)
    im_b = fig2img(fig_b)

    im_np_t = np.array(im_t)
    im_np_t = np.rot90(im_np_t, 1)
    im_np_t = im_np_t[::-1]
    im_np_t = im_np_t[136:520, 48:432, :]

    im_np_a = np.array(im_a)
    im_np_a = np.rot90(im_np_a, 1)
    im_np_a = im_np_a[::-1]
    im_np_a = im_np_a[136:520, 48:432, :]

    im_np_b = np.array(im_b)
    im_np_b = np.rot90(im_np_b, 1)
    im_np_b = im_np_b[::-1]
    im_np_b = im_np_b[136:520, 48:432, :]

    added_img = im_np_t + im_np_a + im_np_b
    return added_img

In [ ]:
head_maps = {k: topo(v) for k,v in mat_feats.items()}

In [ ]:
def get_sessions_subjs():
    path = "/net2/expData/affective_eeg/mahnob_dataset/Sessions"
    sessions = os.listdir(path)
    subjs = {}
    scores = {}
    for session in sessions:
        mytree = ET.parse(f'{path}/{session}/session.xml')
        myroot = mytree.getroot()
        if myroot[0].attrib['id'] not in subjs:
            subjs[myroot[0].attrib['id']] = []
        if 'feltVlnc' in myroot.attrib:
            subjs[myroot[0].attrib['id']].append(session)
            scores[session] = int(myroot.attrib['feltVlnc'])
    return subjs, scores
subjects, subject_scores = get_sessions_subjs()

In [ ]:
# Concat by subject
subject_data = []
valid_sessions = []
for subject, sessions in subjects.items():
    sessions_used = [session for session in sessions if session in head_maps]
    combined_data = np.array([head_maps[session] for session in sessions_used])
    if len(combined_data) > 0:
        subject_data.append(combined_data)
        valid_sessions.append(sessions_used)

In [ ]:
all_data = np.concatenate(subject_data)[:,:,:,:3]
all_data = np.transpose(all_data, (0,3,1,2))

In [ ]:
import torch
from torchvision.models import vgg16
model = vgg16(weights='DEFAULT')
model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-3])
model.eval()
if torch.cuda.is_available():
    model.to('cuda')

all_vgg_feats = []
for i in range(31): # HACK: len(all_data) / 31
    feat_tensor = torch.tensor(all_data[31*i:(i+1)*31], dtype=torch.float, device="cuda" if torch.cuda.is_available() else "cpu") # S T F
    with torch.no_grad():
        result = model(feat_tensor)
    del feat_tensor # free gpu memory

    result = result.numpy(force=True) # always force back to cpu and np
    all_vgg_feats.extend(result)
all_vgg_feats = np.array(all_vgg_feats)

In [ ]:
count = 0
subject_vgg_data = []
for i in subject_data:
    subject_vgg_data.append(all_vgg_feats[count:count+i.shape[0],:])
    count += i.shape[0]
subject_y_data = [np.array([subject_scores[i] for i in subj]) for subj in valid_sessions]

In [ ]:
test_acc = []
for n_subj in range(len(subject_vgg_data)):
    X_train_vgg = np.concatenate([subject_vgg_data[i] for i in range(len(subject_vgg_data)) if i != n_subj])
    y_train_vgg = np.concatenate([subject_y_data[i] for i in range(len(subject_y_data)) if i != n_subj])
    X_test_vgg = subject_vgg_data[n_subj]
    y_test_vgg = subject_y_data[n_subj]

    # transform y to binary
    y_train = np.array(y_train_vgg >= 5, dtype=np.int8)
    y_test = np.array(y_test_vgg >= 5, dtype=np.int8)
    # print(np.unique(y_train, return_counts=True))
    # print(np.unique(y_test, return_counts=True))
    
    pca = PCA(n_components=30)
    pca.fit(X_train_vgg)
    x_train = pca.transform(X_train_vgg)
    
    # Note: data are _roughly_ balanced, not forcing balanced so using equal priors
    lda = LinearDiscriminantAnalysis(priors=(0.5,0.5))
    lda.fit(x_train, y_train)

    
    x_test = pca.transform(X_test_vgg)
    pred_score = lda.score(x_test, y_test)

    test_acc.append(pred_score)
    
print(np.mean(test_acc), np.std(test_acc))

In [ ]:
from tqdm import tqdm
X_train_vgg = np.concatenate(subject_vgg_data)
y_train_vgg = np.concatenate(subject_y_data)
y_train = np.array(y_train_vgg >= 5, dtype=np.int8)

test_acc = []
for i in tqdm(range(len(y_train))):
    pca = PCA(n_components=30)
    x_train = pca.fit_transform(np.concatenate((X_train_vgg[:i,:], X_train_vgg[i+1:,:])))
    y_train_ = np.concatenate((y_train[:i], y_train[i+1:]))

    # Note: data are _roughly_ balanced, not forcing balanced so using equal priors
    lda = LinearDiscriminantAnalysis(priors=(0.5,0.5))
    lda.fit(x_train, y_train_)

    x_test = pca.transform(np.expand_dims(X_train_vgg[i], axis=0))
    pred_score = lda.score(x_test, [y_train[i]])

    test_acc.append(pred_score)
    
print(np.mean(test_acc), np.std(test_acc))